In [1]:
import zipfile
import requests
from collections import Counter
import re
import math

x = requests.get("http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
open("spam.zip", "wb").write(x.content)

zipfile.ZipFile("spam.zip").extractall("./") # 현재 디렉토리에 집 풀기

with open("SMSSpamCollection.txt", "r") as f:
  data = [line.strip().split("\t") for line in f]

train_size = int(0.8 * len(data))
test_size = len(data) - train_size

train = data[:train_size]
test = data[train_size:]

In [2]:
# 사전확률 계산
count = Counter(s for s,c in train)
prior_spam = count['spam']/(count['spam'] + count['ham'])
prior_ham = count['ham']/(count['spam'] + count['ham'])
print(prior_spam, prior_ham)

0.13500784929356358 0.8649921507064364


In [3]:
# spam_words = [w.group() for s,c in train if s=='spam' for w in re.finditer("[0-9a-z_]+",c.lower())]

# for c in [c for s,c in train if s=='spam']:
#   c_words = re.findall("[0-9a-z_]+",c.lower())
#   spam_words.extend(c_words)

spam_words = Counter(w.group() \
                      for s,c in train if s=='spam'\
                      for w in re.finditer("[0-9a-z_]+",c.lower()))

ham_words = Counter(w.group() \
                      for s,c in train if s=='ham'\
                      for w in re.finditer("[0-9a-z_]+",c.lower()))

num_spam_words = sum(spam_words.values())
num_ham_words = sum(ham_words.values())
num_unique_words = len(set(w.group() for s,c in train for w in re.finditer("[0-9a-z_]+",c.lower())))

def predict(text, k=0.5): # k 는 라플라스 스무딩을 위한 값
  words = re.findall("[0-9a-z_]+" , text.lower())

  denom_spam = (num_spam_words + num_unique_words * k)
  
  spam_score = math.log(prior_spam)
  for w in words:
    spam_score += math.log((spam_words[w] + k) / denom_spam)

  denom_ham = (num_ham_words + num_unique_words * k)

  ham_score = math.log(prior_ham)
  for w in words:
    ham_score += math.log((ham_words[w] + k) / denom_ham)

  return ham_score < spam_score

tp,tn, fp, fn = 0, 0, 0, 0

for s, c in test:
  pred = predict(c)
  if pred and s == 'spam' : tp += 1
  elif pred and s != 'spam' : fp += 1
  elif not pred and s == 'spam' : fn += 1
  else: tn += 1

print("tp: ", tp)
print("tn: ", tn)
print("fp: ", fp)
print("fn: ", fn)

tp:  140
tn:  956
fp:  14
fn:  5


In [4]:
accuracy = (tp + tn) / (tp + tn + fp + fn) # 정확도
precision = tp / (fp + tp) # 내가 스팸이라고 했을때 진짜 스팸일 확률
recall = tp / (tp + fn) #실제 스팸중 내가 맞춘 스팸의 갯수
f1 = 2 / (1/precision + 1/recall)   # precision 과 recall의 조화평균

print("accuracy: ", accuracy)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

accuracy:  0.9829596412556054
precision:  0.9090909090909091
recall:  0.9655172413793104
f1:  0.9364548494983278
